In [1]:
!pip install transformer
!pip install datasets

DEPRECATION: Loading egg at /home/cyberkid/projects/venv/lib/python3.12/site-packages/yolox-0.1.0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
DEPRECATION: Loading egg at /home/cyberkid/projects/venv/lib/python3.12/site-packages/yolox-0.1.0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
data_set = 'dataset.csv'
text_column_name = 'email'
label_column_name='category'

In [3]:
model_name='distilbert-base-uncased'
test_size = 0.2
num_label=2

In [4]:
import pandas as pd
df = pd.read_csv(data_set)
df.head()

,email,category
0,"URL: http://www.newsisfree.com/click/-1,817167...",not-spam
1,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",not-spam
2,Dan Kohn <dan@dankohn.com> writes:\n\n\n\n> Gu...,not-spam
3,wintermute wrote:\n\n>>Anyone know where in Ir...,not-spam
4,"I attended the same conference, and was impres...",not-spam


In [5]:
!pip install evaluate

DEPRECATION: Loading egg at /home/cyberkid/projects/venv/lib/python3.12/site-packages/yolox-0.1.0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [6]:
from bs4 import BeautifulSoup

In [7]:
class Cleaner():
  def __init__(self):
    pass
  def put_line_breaks(self,text):
    text = text.replace('</p>','</p>\n')
    return text
  def remove_html_tags(self,text):
    cleantext = BeautifulSoup(text, "lxml").text
    return cleantext
  def clean(self,text):
    text = self.put_line_breaks(text)
    text = self.remove_html_tags(text)
    return text

In [8]:
cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)

/tmp/ipykernel_98915/4246655815.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  cleantext = BeautifulSoup(text, "lxml").text


In [9]:
df.head()

,email,category,text_cleaned
0,"URL: http://www.newsisfree.com/click/-1,817167...",not-spam,"URL: http://www.newsisfree.com/click/-1,817167..."
1,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",not-spam,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-..."
2,Dan Kohn <dan@dankohn.com> writes:\n\n\n\n> Gu...,not-spam,"Dan Kohn writes:\n\n\n\n> Guys, the Habeas In..."
3,wintermute wrote:\n\n>>Anyone know where in Ir...,not-spam,wintermute wrote:\n\n>>Anyone know where in Ir...
4,"I attended the same conference, and was impres...",not-spam,"I attended the same conference, and was impres..."


In [10]:
from sklearn import preprocessing

In [11]:
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())
df['label'] = le.transform(df[label_column_name].tolist())

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_train,df_test = train_test_split(df,test_size=test_size)

In [14]:
from datasets import Dataset

In [15]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [16]:
from transformers import AutoTokenizer


In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text_cleaned"], truncation=True)

/home/cyberkid/projects/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3036 [00:00<?, ? examples/s]

In [19]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/760 [00:00<?, ? examples/s]

In [20]:
from transformers import AutoModelForSequenceClassification

In [21]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
pip install tf-keras

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /home/cyberkid/projects/venv/lib/python3.12/site-packages/yolox-0.1.0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [23]:
import tensorflow as tf

2024-11-18 11:14:14.898153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731908654.914734   98915 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731908654.918933   98915 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 11:14:14.933714: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
#import tensorflow
#from tensorflow as tf
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

In [30]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [31]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [32]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy = "epoch",
    logging_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    
)


/home/cyberkid/projects/venv/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.264200,0.224233,0.944737
2,0.326200,0.188495,0.963158
3,0.104600,0.166401,0.963158
4,0.100700,0.132410,0.977632
5,0.075600,0.128970,0.977632


TrainOutput(global_step=1900, training_loss=0.1742618861951326, metrics={'train_runtime': 1675.6713, 'train_samples_per_second': 9.059, 'train_steps_per_second': 1.134, 'total_flos': 1988002415526816.0, 'train_loss': 0.1742618861951326, 'epoch': 5.0})